In [1]:
from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import prodimopy.read as pread
import pickle
import numpy as np
import pandas as pd
import time
import warnings
from astropy.convolution import convolve_fft
from astropy.convolution import Gaussian1DKernel
import glob
warnings.simplefilter("ignore", DeprecationWarning)

In [3]:
wlrange = [4.9, 28.1]
species = ['C2H2_H', 'CH4_H', 'CO', 'CO2_H', 'HCN_H', 'NH3_H', 'NO_H', 'OH', 'o-H2', 'o-H2O', 'p-H2', 'p-H2O']
ions = ['Ar+', 'Ar++', 'Fe+', 'Ne+', 'Ne++', 'S']
values = np.arange(-0.5, 0.51, 0.25)
data = []
start_time = time.time()
i = 1
for C in values:
    for O in values:
        start_time = time.time()
        
        model_data = []
        
        path = 'models/C_' + str(C) + '_O_' + str(O) + '_CO_*'
        matching_file = glob.glob(path)
        
        CO = matching_file[0].split('CO_')[-1]
        model_name = matching_file[0][7:]
        model_data.append(model_name)
        model_data.append(str(C))
        model_data.append(str(O))
        model_data.append(str(CO))

        model = pread.read_prodimo(matching_file[0], filenameFLiTs='specFLiTs1.out')

        wl, flux, flux_cont = model.FLiTsSpec.convolve(3000, contReturn=True)
        flux -= flux_cont
        model_data.append(wl)
        model_data.append(flux)

        for specie in species:
            model_data.append(model.gen_specFromLineEstimates(ident=specie, wlrange=wlrange, noCont=True, unit='Jy')[1])
        
        total_flux_ion = None
        for ion in ions:
            wl, flux_ion = model.gen_specFromLineEstimates(ident=ion, wlrange=wlrange, noCont=True, unit='Jy')
            if total_flux_ion is None:
                total_flux_ion = flux_ion
            else:
                total_flux_ion += flux_ion
        model_data.append(total_flux_ion)
        model_data.append(wl)
        
        for flits in ['atomic', 'CH4', 'CO', 'CO2', 'H2O', 'HCN', 'NH3', 'NO', 'O2', 'OH']:
            model_flits = pread.read_FLiTs(f'./FLiTs/{flits}/'+model_name, filename=f'specFLiTs1_{flits}.out')
            wl=model_flits.wl
            flux=model_flits.flux-model_flits.flux_cont
            specR = 3000
            
            # Make a new wl grid
            wl_log=np.logspace(np.log10(np.nanmin(wl)),np.log10(np.nanmax(wl)),num=np.size(wl)*1)
            
            # Find stddev of Gaussian kernel for smoothing
            # taken from here https://github.com/spacetelescope/pysynphot/issues/78
            R_grid=(wl_log[1:-1]+wl_log[0:-2])/(wl_log[1:-1]-wl_log[0:-2])/2
            sigma=np.median(R_grid)/specR
            if sigma<1:
              sigma=1
            
            # Interpolate on logarithmic grid
            f_log=np.interp(wl_log,wl,flux)
            # f_cont_log=np.interp(wl_log,wl,flux_cont)
            
            # in the idl script this is interpreted as the FWHM,
            # but the convolution routine wants stddev use relation
            # FWHM=2*sqrt(2ln2)*stddev=2.355/stddev
            # this should than be consistent with the result from the
            # ProDiMo idl script
            gauss=Gaussian1DKernel(stddev=sigma/2.355)
            flux_conv=convolve_fft(f_log,gauss)
            # flux_cont_conv=convolve_fft(f_cont_log,gauss)
            
            # Interpolate back on original wavelength grid
            flux_sm=np.interp(wl,wl_log,flux_conv)
            # flux_cont_sm=np.interp(wl,wl_log,flux_cont_conv)
            
            cut=2*int(sigma)
            flux_smc=flux_sm[cut:(len(flux_sm)-cut)]
            # flux_cont_smc=flux_cont_sm[cut:(len(flux_cont_sm)-cut)]
            wlc=wl[cut:(len(wl)-cut)]
            model_data.append(flux_smc)
            model_data.append(wlc)
        
        data.append(model_data)
        
        end_time = time.time()
        clear_output()
        current_time = end_time - start_time
        print(f'{i}/25; {int(current_time // 60)}:{int(current_time % 60)}')
        i += 1


25/25; 2:37


In [5]:
df = pd.DataFrame(data,
                  columns=['Model Name', 'C Value', 'O Value', 'CO Value', 'FLiTs Wavelength', 'FLiTs Flux',
                           'ProDiMo C2H2_H',
                           'ProDiMo CH4_H', 'ProDiMo CO', 'ProDiMo CO2_H', 'ProDiMo HCN_H', 'ProDiMo NH3_H',
                           'ProDiMo NO_H', 'ProDiMo OH', 'ProDiMo o-H2', 'ProDiMo o-H2O', 'ProDiMo p-H2',
                           'ProDiMo p-H2O', 'ProDiMo atomic', 'ProDiMo Wavelength', 'FLiTs atomic Flux',
                           'FLiTs atomic Wavelength', 'FLiTs CH4 Flux', 'FLiTs CH4 Wavelength', 'FLiTs CO Flux',
                           'FLiTs CO Wavelength', 'FLiTs CO2 Flux',
                           'FLiTs CO2 Wavelength', 'FLiTs H2O Flux', 'FLiTs H2O Wavelength', 'FLiTs HCN Flux',
                           'FLiTs HCN Wavelength', 'FLiTs NH3 Flux', 'FLiTs NH3 Wavelength', 'FLiTs NO Flux',
                           'FLiTs NO Wavelength', 'FLiTs O2 Flux', 'FLiTs O2 Wavelength', 'FLiTs OH Flux',
                           'FLiTs OH Wavelength'])

with open('ModelData.pkl', 'wb') as f:
    pickle.dump(df, f)
